In [1]:
from narupa.app.client import NarupaImdClient
from narupa.ase.converter import frame_data_to_ase
import numpy as np
import subprocess
import nglview
import shlex
from nglclient import NGLClient

_ColormakerRegistry()

## Run the server

Run the LAMMPS Lammps or OpenMM or anything else. 

In [ ]:
lmp_pid = process.pid
process.terminate()
process.kill()
process.wait()
print("LAMMPS was killed on pid", lmp_pid)

In [ ]:
process = subprocess.Popen(['./lmp'], stdin=open('in.peptide'))
print('Started LAMMPS server')

## Start an IMD client

In [2]:
class NGLClient(NarupaImdClient):
    def __init__(self, dynamic_bonds=False, *args, update_callback=None, **kwargs):
        self._view = None
        super().__init__(*args, **kwargs)
        self.update_callback = update_callback
        self.dynamic_bonds = dynamic_bonds
    
    @property
    def view(self):
        if self._view is None or self.dynamic_bonds:
            atoms = frame_data_to_ase(
                self.first_frame,
                topology=True,
                positions=False,
            )
            atoms.set_positions(
                np.array(self.latest_frame.particle_positions) * 10
            )
            self._view = nglview.show_ase(atoms)
        return self._view

    def _on_frame_received(self, frame_index:int, frame):
        super()._on_frame_received(frame_index, frame)
        self.view.set_coordinates(
            {0: np.array(self.latest_frame.particle_positions) * 10}
        )
        if self.update_callback is not None:
            self.update_callback(self.universe)


def _get_structure_string(write_method, suffix='.pdb'):
    fname = 'tmp.pdb'
    write_method(fname)
    with open(fname) as fh:
        return fh.read()

nglview.adaptor._get_structure_string = _get_structure_string

In [2]:
client = NGLClient(address='localhost', dynamic_bonds=False, trajectory_port=54321,imd_port=54322)


In [3]:
atoms = frame_data_to_ase(client.first_frame, topology=True, positions=False)
atoms.set_positions(np.array(client.latest_frame.particle_positions) * 10)

In [4]:
client.view


NGLWidget()

In [ ]:
view = nglview.show_pdbid("3pqr")
view

In [ ]:
## Start a spring interaction

In [6]:
from narupa.imd.particle_interaction import ParticleInteraction
interaction = ParticleInteraction("5", "0", position=(0,0,0), particles=(68), scale=10, interaction_type="spring")
interaction_id = client.start_interaction()
client.update_interaction(interaction_id, interaction)

In [ ]:
client.stop_interaction(interaction_id)

In [8]:
client.close()